In [1]:
!pip install dgl==1.1.3+cu121 -f https://data.dgl.ai/wheels/cu121/repo.html

Looking in links: https://data.dgl.ai/wheels/cu121/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.4 MB/s eta 0:00:00


In [2]:
!pip install torchdata

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 1.6 MB/s eta 0:00:00


In [3]:
import dgl
import torch
import torchdata
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
import torch
torch.__version__

import dgl.data

dataset = dgl.data.KarateClubDataset()
g = dataset[0]
# Split edge set for training and testing
u, v = g.edges()

eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']

import torch

num_nodes = g.num_nodes()  # Số lượng nút trong đồ thị
g.ndata['feat'] = torch.randn(num_nodes, 16)  # Gán đặc trưng ngẫu nhiên với kích thước 16

print("Node features added to g:", g.ndata['feat'])

train_g = dgl.remove_edges(g, eids[:test_size])

Node features added to g: tensor([[ 1.0309e+00, -1.7742e+00,  8.3869e-01,  8.2744e-01, -1.8814e-01,
         -1.7116e-01,  6.2765e-01, -1.4119e+00,  7.5390e-01, -4.2385e-01,
          5.6343e-01,  2.0219e-01,  1.1149e+00,  8.1149e-01,  1.5979e+00,
          7.6580e-01],
        [ 6.0932e-01, -1.0783e+00, -7.1308e-01, -8.0459e-01,  7.0423e-01,
          4.8890e-01,  1.3845e+00, -6.6593e-01, -1.5444e+00,  5.6742e-01,
         -1.1319e+00, -7.2154e-01,  6.2415e-01,  4.9152e-02,  2.4487e+00,
          7.7661e-01],
        [ 3.0962e-01, -1.8426e+00, -8.5116e-01, -1.9749e+00, -4.8951e-01,
          2.3210e+00,  8.6983e-01,  4.6535e-01, -5.7577e-01, -2.0288e-01,
         -1.1821e+00,  2.3041e-01,  1.6146e-01,  9.5654e-01,  1.5897e+00,
         -3.2411e-01],
        [ 4.6420e-01,  6.3984e-01,  7.0732e-01,  2.5268e-02, -1.4145e+00,
         -3.2638e-01, -1.5565e+00, -5.6746e-02, -7.5860e-01,  4.0207e-01,
          4.2582e-01,  5.6658e-01, -1.2186e+00, -3.3936e-01, -8.4144e-01,
          4.7240e

In [5]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))


# Thumbnail credits: Link Prediction with Neo4j, Mark Needham
# sphinx_gallery_thumbnail_path = '_static/blitz_4_link_predict.png'

In epoch 0, loss: 19.41141128540039
In epoch 5, loss: 4.169886589050293
In epoch 10, loss: 1.9670662879943848
In epoch 15, loss: 1.1372506618499756
In epoch 20, loss: 0.8657136559486389
In epoch 25, loss: 0.7069770693778992
In epoch 30, loss: 0.5888493657112122
In epoch 35, loss: 0.5001443028450012
In epoch 40, loss: 0.4425227642059326
In epoch 45, loss: 0.401736319065094
In epoch 50, loss: 0.36754506826400757
In epoch 55, loss: 0.3358902037143707
In epoch 60, loss: 0.31068047881126404
In epoch 65, loss: 0.2904728055000305
In epoch 70, loss: 0.2726384699344635
In epoch 75, loss: 0.25692176818847656
In epoch 80, loss: 0.242902010679245
In epoch 85, loss: 0.2297246903181076
In epoch 90, loss: 0.21736665070056915
In epoch 95, loss: 0.20529979467391968
AUC 0.8


In [ ]:
**AUC (GraphSAGE chiếm ưu thế, nhưng vẫn cần cải thiện):**

**GraphSAGE đạt AUC = 0.8**, cao hơn hầu hết các phương pháp truyền thống nhưng chưa vượt trội so với kỳ vọng của GNN.

**Quá trình huấn luyện:**
- Từ **epoch 0** với loss là 19.41, mô hình đã có sự giảm đáng kể qua các epoch, chỉ còn **0.205** ở **epoch 95**.
- Điều này cho thấy mô hình đã học tốt qua thời gian, với quá trình tối ưu liên tục và ổn định.

**Random Forest:**
- Kết quả của **Random Forest** (AUC = 0.778) vẫn sát sao, cho thấy nó là một đối thủ cạnh tranh khá mạnh dù không tận dụng được cấu trúc đồ thị như GraphSAGE.

**Phương pháp truyền thống:**
- Các phương pháp như **Common Neighbors**, **Jaccard Coefficient**, và **Adamic-Adar** chỉ đạt AUC từ 0.4 - 0.5, cho thấy khả năng dự đoán tương đương với một mô hình ngẫu nhiên.
- **Preferential Attachment** có AUC = 0.702, tốt hơn nhưng đánh đổi Precision để có Recall cao.

**GraphSAGE (GNN):**
- Mặc dù đạt AUC = 0.8, GraphSAGE vẫn chưa thực sự thể hiện được sức mạnh vượt trội như kỳ vọng của GNN trong bài toán này.
- Quá trình giảm loss ổn định gợi ý rằng, với sự tối ưu thêm (tăng số lượng epoch, tinh chỉnh siêu tham số, hoặc tăng dữ liệu huấn luyện), AUC có thể còn cải thiện hơn.

**Kết luận:**
- Dù **GraphSAGE** đang dẫn đầu trong bài toán này, khoảng cách giữa AUC của GNN và Random Forest là khá nhỏ.
- Điều này chỉ ra rằng cần có những cải tiến để khai thác tối đa tiềm năng của GNN trong bài toán dự đoán liên kết trên mạng xã hội.

In [ ]:
GMạng Nơ-ron Đồ Thị (Graph Neural Network - GNN) là một loại mô hình học máy được thiết kế đặc biệt để làm việc với dữ liệu đồ thị.
GNN có khả năng mở rộng và áp dụng trên các đồ thị có cấu trúc phức tạp, như mạng xã hội, mạng lưới giao thông, hay bất kỳ hệ thống nào có mối quan hệ giữa các đối tượng.
GNN hoạt động bằng cách truyền thông tin qua các đỉnh và cạnh trong đồ thị.
Mô hình học thông qua việc cập nhật và kết hợp thông tin từ các hàng xóm của mỗi đỉnh, cho phép nắm bắt thông tin cấu trúc và tương tác giữa các đối tượng trong đồ thị.
Một trong những đặc điểm đáng chú ý của GNN là khả năng tích hợp thông tin từ cả đặc trưng của các đỉnh và cấu trúc đồ thị.
Điều này cho phép GNN học mô hình phức tạp và biểu diễn các mối quan hệ phức tạp giữa các đối tượng trong đồ thị.
GNN đã chứng tỏ được hiệu quả trong nhiều nhiệm vụ, bao gồm phân loại đồ thị, phân loại nút, dự đoán liên kết và nhúng đồ thị.
Các ứng dụng của GNN rất đa dạng, từ phân tích mạng xã hội, gợi ý người dùng, cho đến phát hiện và kiểm soát các hiện tượng trong các hệ thống phức tạp.

Thành phần cơ bản của

Đỉnh (Nodes): Đại diện cho các thực thể, chẳng hạn như người trong mạng xã hội, trang web, hoặc sản phẩm.
Cạnh (Edges): Đại diện cho mối quan hệ hoặc tương tác giữa các thực thể, ví dụ: tình bạn, liên kết web, hoặc giao dịch.
Đặc trưng (Features): Mỗi đỉnh hoặc cạnh có thể có các đặc trưng riêng, như độ tuổi của một người hoặc trọng số của một cạnh.

Nguyên lý hoạt động

GNN hoạt động dựa trên việc tổng hợp thông tin từ các đỉnh lân cận, giúp một đỉnh hiểu thêm về ngữ cảnh của nó trong toàn đồ thị. Quá trình này thường được lặp qua nhiều bước (layers):

Message Passing (Truyền thông điệp): Mỗi đỉnh nhận thông tin từ các đỉnh láng giềng thông qua các cạnh.
Aggregation (Tổng hợp): Tóm tắt thông tin từ các đỉnh láng giềng (ví dụ: tính trung bình, tổng hoặc dùng hàm học được).
Update (Cập nhật): Cập nhật trạng thái hoặc đặc trưng của đỉnh dựa trên thông tin tổng hợp.

Các loại GNN phổ biến
GCN (Graph Convolutional Network): Dựa trên phép toán tích chập trên đồ thị, hiệu quả với dữ liệu đồ thị nhỏ hoặc vừa.
GraphSAGE: Học cách tổng hợp thông tin từ láng giềng mẫu, phù hợp với đồ thị lớn.
GAT (Graph Attention Network): Sử dụng cơ chế attention để gán trọng số khác nhau cho láng giềng quan trọng.
GIN (Graph Isomorphism Network): Tăng cường khả năng phân biệt giữa các đồ thị khác nhau.
Graph Autoencoders (GAE): GAE là một dạng GNN được sử dụng để học biểu diễn nhúng (embedding) của đồ thị. GAE cố gắng tái tạo đồ thị gốc từ biểu diễn nhúng, giúp học các đặc trưng ngầm của đồ thị và khám phá cấu trúc ẩn.
Graph Recurrent Neural Networks (GRNN): GRNN sử dụng kiến trúc mạng nơ-ron hồi quy để xử lý dữ liệu đồ thị có thứ tự thời gian.
GRNN có khả năng mô hình các quá trình động trên đồ thị như dự đoán chuỗi thời gian hoặc phân tích dữ liệu đồ thị theo thời gian.

